# Sanger cell line Affymetrix gene expression project

### Source

- From GEO: http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE68950

In [1]:
from pathlib import Path
import pandas as pd

In [2]:
# Setup logging 
import logging
logger = logging.getLogger()
logger.handlers[0].setFormatter(
    logging.Formatter(
        fmt=(
            "%(asctime)s.%(msecs)03d %(levelname).1s "
            "[%(name)s] %(message)s"
        ),
        datefmt="%y-%m-%d %H:%M:%S",
    )
)
logger.setLevel(logging.INFO)

In [3]:
logger.info('Logger works!')

16-01-05 21:20:14.483 I [root] Logger works!


In [4]:
data_root = Path('../').resolve()
raw_data_root = data_root.joinpath(
    'raw', 
    'Sanger_Cell_Line_Project_Affymetrix_QCed_Data_n798/'
)
logger.info(
    'Required paths exist: %s' % 
    all(map((lambda p: p.exists()), [data_root, raw_data_root]))
)

16-01-05 21:20:14.487 I [root] Required paths exist: True


In [5]:
sample_info_p = next(raw_data_root.glob('*.xls'))
sample_raw_data_p = next(raw_data_root.glob('*.tsv.gz'))
plat_info = next(data_root.glob('*.csv'))

print('Sample info:', sample_info_p)
print('Sample raw data:', sample_raw_data_p)
print('Platform info:', plat_info)

Sample info: /Users/liang/code/dj_carkinos/data/raw/Sanger_Cell_Line_Project_Affymetrix_QCed_Data_n798/Sanger_affy_n798_sample_info_published.xls
Sample raw data: /Users/liang/code/dj_carkinos/data/raw/Sanger_Cell_Line_Project_Affymetrix_QCed_Data_n798/Sanger_Cell_Line_Project_Affymetrix_QCed_Data_n798.gct.tsv.gz
Platform info: /Users/liang/code/dj_carkinos/data/Affy_U133A_probe_info.csv


## Read in Platform Annotation

The platform annotation is created by `Get_Affy_probes.R`

In [6]:
plat_info_df = pd.read_csv(plat_info.as_posix())

In [7]:
plat_info_df.head()

,PROBEID,SYMBOL,ENTREZID,GENENAME
0,1007_s_at,DDR1,780,discoidin domain receptor tyrosine kinase 1
1,1007_s_at,MIR4640,100616237,microRNA 4640
2,1053_at,RFC2,5982,"replication factor C (activator 1) 2, 40kDa"
3,117_at,HSPA6,3310,heat shock 70kDa protein 6 (HSP70B')
4,121_at,PAX8,7849,paired box 8


## Read in Sample Info

In [8]:
sample_info_df = pd.read_excel(sample_info_p.as_posix())

In [9]:
sample_info_df = sample_info_df.loc[
    :, ['Scan', 'SampleName', 'PrimarySite', 'PrimaryHist']
]
sample_info_df.columns = [
    'name', 'cell_line', 'primary_site', 'primary_histology'
]

In [10]:
sample_info_df['filename'] = sample_info_df.name + '.CEL'

In [11]:
# Using 800 samples take up 1.1GB SQLite database, which is too large
sample_info_df = sample_info_df.head(n=100)

In [12]:
sample_info_df.head()

,name,cell_line,primary_site,primary_histology,filename
0,5500024035100021608461.G01,380,haematopoietic and lymphoid tissue,lymphoid neoplasm,5500024035100021608461.G01.CEL
1,5500024034290101707049.A01,697,haematopoietic and lymphoid tissue,haematopoietic neoplasm,5500024034290101707049.A01.CEL
2,5500024052603032009483.A09,5637,urinary tract,carcinoma,5500024052603032009483.A09.CEL
3,5500024035100021608461.H01,22RV1,prostate,carcinoma,5500024035100021608461.H01.CEL
4,5500024032848101507998.D02,23132-87,stomach,carcinoma,5500024032848101507998.D02.CEL


In [13]:
# To make sure a cell_line name only has one primary_site and primary_histology 
for grp, df in sample_info_df.iloc[:, 1:-1].groupby('cell_line'):
    assert len(pd.unique(df.to_records(index=False))) == 1

## Read Sample Raw Data

In [14]:
sample_raw_data_df = pd.read_table(sample_raw_data_p.as_posix(), skiprows=2)

In [15]:
sample_raw_data_df.head()

,NAME,DESC,5500024030401071707289.A01.CEL,5500024030401071707289.A02.CEL,5500024030401071707289.A03.CEL,5500024030401071707289.A04.CEL,5500024030401071707289.A05.CEL,5500024030401071707289.A06.CEL,5500024030401071707289.A07.CEL,5500024030401071707289.A08.CEL,...,5500024052861011409506.H03.CEL,5500024052861011409506.H04.CEL,5500024052861011409506.H05.CEL,5500024052861011409506.H06.CEL,5500024052861011409506.H07.CEL,5500024052861011409506.H08.CEL,5500024052861011409506.H09.CEL,5500024052861011409506.H10.CEL,5500024052861011409506.H11.CEL,5500024052861011409506.H12.CEL
0,1007_s_at,NaN,2074,94,926,1587,762,466,116,599,...,738,113,717,687,347,88,1425,877,163,247
1,1053_at,NaN,120,272,160,144,304,314,341,260,...,149,186,73,85,237,366,160,144,160,362
2,117_at,NaN,22,140,20,18,18,21,43,28,...,25,20,21,30,26,31,24,20,25,344
3,121_at,NaN,62,49,54,87,903,57,98,71,...,1690,70,1470,69,74,76,104,73,57,55
4,1255_g_at,NaN,15,16,16,16,15,15,16,15,...,17,15,17,16,23,17,17,17,18,24


## Setup Django

Since Django needs to load the modules by relative import, change the working directory to `/src`.

In [16]:
cd "../../src/"

/Users/liang/code/dj_carkinos/src


In [17]:
import django
import os

In [18]:
os.environ['DJANGO_SETTINGS_MODULE'] = 'carkinos.settings.local'

In [19]:
django.setup()

In [20]:
from datasets.models import (
    CellLine, DataSet, MicroarrayPlatform, 
    Sample, ProbeValue, ProbeInfo
)

In [21]:
logging.getLogger('django.db.backends').setLevel(logging.INFO)

### Setup Microarray Plaform

In [22]:
# Get all probes
probe_list = '\n'.join((sample_raw_data_df['NAME'].values))

In [23]:
platform = MicroarrayPlatform(
    name='GPL3921',
    manufacturer='Affymetrix',
    description='Affymetrix HT Human Genome U133A Array',
    probe_list=probe_list,
)

In [24]:
platform.save()

In [25]:
platform

<MicroarrayPlatform: GPL3921 (GPL3921)>

### Platform Probe Info

In [26]:
plat_info_df.SYMBOL.fillna('', inplace=True)
plat_info_df.GENENAME.fillna('', inplace=True)

In [27]:
plat_info_df.ENTREZID.fillna(0, inplace=True)

In [28]:
plat_info_df.tail()

,PROBEID,SYMBOL,ENTREZID,GENENAME
24544,AFFX-r2-Hs28SrRNA-3_at,,0,
24545,AFFX-r2-Hs28SrRNA-5_at,,0,
24546,AFFX-r2-Hs28SrRNA-M_at,,0,
24547,AFFX-r2-P1-cre-3_at,,0,
24548,AFFX-r2-P1-cre-5_at,,0,


In [29]:
plat_info_df.iloc[927]

PROBEID     201265_at
SYMBOL               
ENTREZID            0
GENENAME             
Name: 927, dtype: object

In [30]:
probes = []
for rowid, row in plat_info_df.iterrows():
    probes.append(ProbeInfo(
        identifier=row.PROBEID,
        platform=platform,
        gene_symbol=row.SYMBOL,
        gene_name=row.GENENAME,
        entrez_id=(row.ENTREZID if row.ENTREZID > 0 else None),
    ))

In [31]:
len(probes)

24549

In [32]:
ProbeInfo.objects.bulk_create(probes)
print(ProbeInfo.objects.count())

24549


### Setup Dataset

In [33]:
dset = DataSet(
    name = 'Sanger Cell Line Project',
    data_path = '',
)

In [34]:
dset.save()

In [35]:
dset

<DataSet: Sanger Cell Line Project>

### Setup Cell Lines, Samples and Their Values

In [36]:
sample_info_df.head()

,name,cell_line,primary_site,primary_histology,filename
0,5500024035100021608461.G01,380,haematopoietic and lymphoid tissue,lymphoid neoplasm,5500024035100021608461.G01.CEL
1,5500024034290101707049.A01,697,haematopoietic and lymphoid tissue,haematopoietic neoplasm,5500024034290101707049.A01.CEL
2,5500024052603032009483.A09,5637,urinary tract,carcinoma,5500024052603032009483.A09.CEL
3,5500024035100021608461.H01,22RV1,prostate,carcinoma,5500024035100021608461.H01.CEL
4,5500024032848101507998.D02,23132-87,stomach,carcinoma,5500024032848101507998.D02.CEL


In [37]:
# Create all cell lines
known_cell_lines = {}
for rowid, row in sample_info_df.iterrows():
    cl = row.cell_line
    try:
        cell_line = known_cell_lines[cl]
    except KeyError:
        cell_line = CellLine(
            name=str(row.cell_line), 
            primary_site=row.primary_site, 
            primary_histology=row.primary_histology
        )
        cell_line.save()
        known_cell_lines[cl] = cell_line

In [38]:
sample_raw_data_df.index = sample_raw_data_df.NAME

In [39]:
# Create all samples and fetch their probe values
for rowid, row in sample_info_df.iterrows():
    name = row['name']
    logger.info('Writing data of Sample %s ...' % name)
    # Create sample
    sample = Sample(
        name=name,
        filename=row.filename,
        cell_line=known_cell_lines[row.cell_line],
        dataset=dset,
        platform=platform
    )
    # Setup probe values
    if sample.filename not in sample_raw_data_df.columns:
        logger.warning('Sample %s does not have raw data, skipped')
        continue
    sample.save()   
    sample_data = sample_raw_data_df.loc[:, sample.filename]
    probe_vals = []
    for ix, val in sample_data.iteritems():
        pv = ProbeValue(
            sample=sample,
            probe_id=ix,
            value=val
        )
        probe_vals.append(pv)
    ProbeValue.objects.bulk_create(probe_vals)

16-01-05 13:20:22.702 I [root] Writing data of Sample 5500024035100021608461.G01 ...
16-01-05 13:20:23.831 I [root] Writing data of Sample 5500024034290101707049.A01 ...
16-01-05 13:20:24.966 I [root] Writing data of Sample 5500024052603032009483.A09 ...
16-01-05 13:20:26.102 I [root] Writing data of Sample 5500024035100021608461.H01 ...
16-01-05 13:20:27.223 I [root] Writing data of Sample 5500024032848101507998.D02 ...
16-01-05 13:20:28.346 I [root] Writing data of Sample 5500024030401071707289.D04 ...
16-01-05 13:20:29.469 I [root] Writing data of Sample 5500024030401071707289.C10 ...
16-01-05 13:20:30.589 I [root] Writing data of Sample 5500024052861011409506.D05 ...
16-01-05 13:20:31.713 I [root] Writing data of Sample 5500024032848101507998.E02 ...
16-01-05 13:20:32.843 I [root] Writing data of Sample 5500024052861011409506.E01 ...
16-01-05 13:20:33.982 I [root] Writing data of Sample 5500024034290101707049.D01 ...
16-01-05 13:20:35.125 I [root] Writing data of Sample 55000240328